# Predicting Lending Club Interest Rates with Linear Regression


## <font color="red">Important -- Read This First!"</font>

If you cloned this notebook from GitHub, you will need to either <br>
1. Download the *loan.csv* file from Kaggle (link given under [Datasets](#Datasets) or <br> 
2. Unzip the *LCloan.csv.bz3* from the [data](./data) folder.

If you download from Kaggle, you will need to rename *loan.csv* to *LCloan.csv* and save it in the [data folder](./data).

## Project Description

We are going to explore anonymous loan data provided by LendingClub. <br>
<font color="darkgreen">We'll try to predict the interest rate for  loan applications based on the data provided.</font> 

To do this requires us to create a model. One of the simplest models we can create is a Linear Model where we start with the assumption that a dependent variable (e.g., interest rates) varies linearly with the independent variable(s). Essentially fitting a straight line through the data and expecting it to give us a good prediction for values we haven't seen.

In building the model we want to minimize the error so that when we make a new prediction we can do so with utmost confidence (~95% is a good benchmark). There are multiple ways of minimizing this error, simplest being the least-squares method. In other words, calculating the sum of squares of each error (to eliminate negatives) and minimizing this number.


#### About Lending Club

The [Lending Club](https://www.lendingclub.com/) is an online marketplace for personal loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return. Each borrower fills out a comprehensive application, providing their past financial history, the reason for the loan, and more. Lending Club evaluates each borrower’s credit score using past historical data and assigns an interest rate to the borrower. 

The benefit to us is that some of these data have been made available to us for analysis. While loan data excludes personally identifiable information, it does include attributes like FICO score, location, annual income, lines of credit, and descriptions of why the applicant needs the loan.



## Datasets

There are several sources of Lending Club data that you might be able to use to test the code:
- [Kaggle option 1](https://www.kaggle.com/wendykan/lending-club-loan-data)
- [Kaggle option 2](https://www.kaggle.com/wordsforthewise/lending-club)
- [Data World](https://data.world/jaypeedevlin/lending-club-loan-data-2007-11) (data from 2007)
- [Lending Club](https://www.lendingclub.com/auth/login?login_url=%2Finfo%2Fdownload-data.action)

Here we use the first Kaggle dataset (option 1).

These data contain complete loan data for all loans issued through the 2007-2015, including the current loan status (Current, Late, Fully Paid, etc.) and latest payment information. The file containing loan data through the "present" contains complete loan data for all loans issued through the previous completed calendar quarter. Additional features include credit scores, number of finance inquiries, address including zip codes, and state, and collections among others. A [data dictionary](./data/LCDataDictionary.xlsx) is provided in a separate file.

In [1]:
import pandas as pd
import numpy as np

Let's first download data to a pandas df from the loan csv file. 


In [2]:


# This is a large dataset so it might take a minute or two to load.
loan=pd.read_csv('./data/LCloan.csv', parse_dates=True, low_memory=False) 

#Backup of the dataframe so we don't have to download data everytime
loanbk=loan.copy()


Let's look at the DataFrame, dimensions and contents

In [3]:
loan.shape

(2260668, 145)

In [4]:
loan.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,D1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,4000,4000,4000.0,36 months,18.94,146.51,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,30000,30000,30000.0,60 months,16.14,731.78,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


Let's checkout the data in the dataframe

In [5]:
loan.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,url,dti,...,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
count,0.0,0.0,2.260668e+06,2.260668e+06,2.260668e+06,2.260668e+06,2.260668e+06,2.260664e+06,0.0,2.258957e+06,...,10613.0,10613.000000,10613.0,10613.000000,8426.000000,10613.000000,10613.000000,33056.000000,33056.000000,33056.000000
mean,NaN,NaN,1.504693e+04,1.504166e+04,1.502344e+04,1.309291e+01,4.458076e+02,7.799243e+04,NaN,1.882420e+01,...,3.0,155.006696,3.0,13.686422,454.840802,11628.036442,193.606331,5030.606922,47.775600,13.148596
std,NaN,NaN,9.190245e+03,9.188413e+03,9.192332e+03,4.832114e+00,2.671737e+02,1.126962e+05,NaN,1.418333e+01,...,0.0,129.113137,0.0,9.728138,375.830737,7615.161123,198.694368,3692.027842,7.336379,8.192319
min,NaN,NaN,5.000000e+02,5.000000e+02,0.000000e+00,5.310000e+00,4.930000e+00,0.000000e+00,NaN,-1.000000e+00,...,3.0,0.640000,3.0,0.000000,1.920000,55.730000,0.010000,44.210000,0.200000,0.000000
25%,NaN,NaN,8.000000e+03,8.000000e+03,8.000000e+03,9.490000e+00,2.516500e+02,4.600000e+04,NaN,1.189000e+01,...,3.0,59.370000,3.0,5.000000,174.967500,5628.730000,43.780000,2227.000000,45.000000,6.000000
50%,NaN,NaN,1.290000e+04,1.287500e+04,1.280000e+04,1.262000e+01,3.779900e+02,6.500000e+04,NaN,1.784000e+01,...,3.0,119.040000,3.0,15.000000,352.605000,10044.220000,132.890000,4172.855000,45.000000,14.000000
75%,NaN,NaN,2.000000e+04,2.000000e+04,2.000000e+04,1.599000e+01,5.933200e+02,9.300000e+04,NaN,2.449000e+01,...,3.0,213.260000,3.0,22.000000,622.792500,16114.940000,284.180000,6870.782500,50.000000,18.000000
max,NaN,NaN,4.000000e+04,4.000000e+04,4.000000e+04,3.099000e+01,1.719830e+03,1.100000e+08,NaN,9.990000e+02,...,3.0,943.940000,3.0,37.000000,2680.890000,40306.410000,1407.860000,33601.000000,521.350000,181.000000


Let's do some data cleaning

Drop columns without any entries

In [6]:
loan.dropna(axis=1, how='all', inplace=True)
loan.shape

(2260668, 142)

Let's look at a list of the column names that are left

In [7]:
loan.columns.values.tolist()

['loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_

We don't need all these columns, let us drop some

In [8]:
loan.drop(['grade', 'sub_grade', 'emp_title', 'issue_d', 'pymnt_plan', 
                'desc', 'title', 'initial_list_status', 'last_pymnt_d', 'last_pymnt_amnt', 
                'next_pymnt_d', 'last_credit_pull_d', 'policy_code', 'emp_length', 'addr_state',
                'zip_code'], axis=1, inplace=True)
loan.shape

(2260668, 126)

Okay, we've reduced the DataFrame to 126 columns.  But we can still tidy up some more.

Since we want to do a linear regression, it's nice to know which of our attributes are numerical vs. categorical (non-numeric).

In [9]:
loan.dtypes

loan_amnt                  int64
funded_amnt                int64
funded_amnt_inv          float64
term                      object
int_rate                 float64
                          ...   
settlement_status         object
settlement_date           object
settlement_amount        float64
settlement_percentage    float64
settlement_term          float64
Length: 126, dtype: object

Looking at types in the loan dataframe, we see ints, floats which represent our numeric data types.  Object represents our categorical data types.

Next we look more carefully at which data are categorical.

In [10]:
loan.select_dtypes(include=['object']).head()

,term,home_ownership,verification_status,loan_status,purpose,earliest_cr_line,application_type,verification_status_joint,sec_app_earliest_cr_line,hardship_flag,...,hardship_status,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_loan_status,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date
0,36 months,RENT,Not Verified,Current,debt_consolidation,Apr-2001,Individual,NaN,NaN,N,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
1,60 months,MORTGAGE,Source Verified,Current,debt_consolidation,Jun-1987,Individual,NaN,NaN,N,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
2,36 months,MORTGAGE,Source Verified,Current,debt_consolidation,Apr-2011,Individual,NaN,NaN,N,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
3,36 months,MORTGAGE,Source Verified,Current,debt_consolidation,Feb-2006,Individual,NaN,NaN,N,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
4,60 months,MORTGAGE,Not Verified,Current,debt_consolidation,Dec-2000,Individual,NaN,NaN,N,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN


Break view into chunks so we can see all the columns

In [11]:
loan[loan['verification_status_joint'].notnull()].iloc[:5,0:50].select_dtypes(include=['object'])

,term,home_ownership,verification_status,loan_status,purpose,earliest_cr_line,application_type,verification_status_joint
15,36 months,MORTGAGE,Verified,Current,debt_consolidation,Dec-1988,Joint App,Verified
17,60 months,MORTGAGE,Not Verified,Current,debt_consolidation,Oct-2010,Joint App,Not Verified
23,36 months,MORTGAGE,Source Verified,Current,car,Oct-2004,Joint App,Source Verified
25,60 months,MORTGAGE,Source Verified,Current,debt_consolidation,Oct-1999,Joint App,Source Verified
39,60 months,MORTGAGE,Not Verified,Current,credit_card,Dec-1998,Joint App,Not Verified


In [12]:
loan[loan['hardship_reason'].notnull()].iloc[:5,50:111].select_dtypes(include=['object'])

,sec_app_earliest_cr_line,hardship_flag,hardship_type,hardship_reason,hardship_status,hardship_start_date,hardship_end_date
95385,Aug-2007,Y,INTEREST ONLY-3 MONTHS DEFERRAL,UNEMPLOYMENT,ACTIVE,Feb-2019,Apr-2019
130968,Apr-2003,N,INTEREST ONLY-3 MONTHS DEFERRAL,NATURAL_DISASTER,COMPLETED,Oct-2018,Dec-2018
132095,NaN,N,INTEREST ONLY-3 MONTHS DEFERRAL,NATURAL_DISASTER,COMPLETED,Oct-2018,Dec-2018
148481,NaN,N,INTEREST ONLY-3 MONTHS DEFERRAL,NATURAL_DISASTER,COMPLETED,Oct-2018,Dec-2018
192074,NaN,N,INTEREST ONLY-3 MONTHS DEFERRAL,NATURAL_DISASTER,COMPLETED,Nov-2018,Jan-2019


In [13]:
loan[loan['hardship_loan_status'].notnull()].iloc[:5,111:].select_dtypes(include=['object'])

,payment_plan_start_date,hardship_loan_status,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date
95385,Feb-2019,Late (16-30 days),Cash,N,NaN,NaN,NaN
130968,Oct-2018,Issued,Cash,N,NaN,NaN,NaN
132095,Oct-2018,Issued,Cash,N,NaN,NaN,NaN
148481,Oct-2018,Issued,Cash,N,NaN,NaN,NaN
192074,Nov-2018,Current,Cash,N,NaN,NaN,NaN


In [14]:
loan[loan['settlement_date'].notnull()].iloc[:5,111:].select_dtypes(include=['object'])

,payment_plan_start_date,hardship_loan_status,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date
92796,NaN,NaN,Cash,Y,Feb-2019,ACTIVE,Feb-2019
116624,NaN,NaN,Cash,Y,Feb-2019,ACTIVE,Feb-2019
143914,NaN,NaN,Cash,Y,Feb-2019,ACTIVE,Feb-2019
155735,NaN,NaN,Cash,Y,Dec-2018,ACTIVE,Dec-2018
182418,NaN,NaN,Cash,Y,Feb-2019,ACTIVE,Feb-2019


Looking at this, we notice a few things...

- Dates are listed as Month-Year, where month is a string and year is a number
- the attribute *term* is a number followed by the string months


We can remove the string "months" from the loan term and the month preceeding the year in the date columns.

First we create a list of the columns using the Month-Year format

In [15]:
month_date_ls = ['earliest_cr_line','sec_app_earliest_cr_line','hardship_start_date','hardship_end_date',
                'payment_plan_start_date','debt_settlement_flag_date','settlement_date']

Then use regex to remove the Month part of the string

For more on using regex, see here:<br>
* [regex](https://www.ntu.edu.sg/home/ehchua/programming/howto/Regexe.html)
* [regex101.com](https://regex101.com/)

For more on approaches of replacing strings in a pandas DataFrame column, see here:<br>
* [Remove partial string from dataframe with Pandas](https://stackoverflow.com/questions/50731702/remove-partial-string-from-dataframe-with-pandas)
* [Replace a string with NaN](https://stackoverflow.com/questions/53668421/replace-a-string-value-with-nan-in-pandas-data-frame-python)
* [5 methods to remove the string from your data in Python](https://towardsdatascience.com/5-methods-to-remove-the-from-your-data-in-python-and-the-fastest-one-281489382455)
* [List comprehension](https://stackoverflow.com/questions/4406389/if-else-in-a-list-comprehension)

In [16]:
# import regex

import re

Get rid of Month- in Month-Year format

In [17]:
for col in month_date_ls:
    loan[col].fillna('nan', inplace=True)
    p = re.compile(r'[0-9]+.')
    loan[col] = [p.search(date)[0] if p.search(date) is not None else "nan"
                         for date in loan[col].tolist()] 
    loan[col] = loan[col].replace('nan', np.nan)

Get rid of *months* in term column

In [18]:
loan.term=loan.term.str.split(' ',2).str[1]

Let's check the output to make sure it worked

In [19]:
loan[loan['verification_status_joint'].notnull()].iloc[:5,0:50].select_dtypes(include=['object'])

,term,home_ownership,verification_status,loan_status,purpose,earliest_cr_line,application_type,verification_status_joint
15,36,MORTGAGE,Verified,Current,debt_consolidation,1988,Joint App,Verified
17,60,MORTGAGE,Not Verified,Current,debt_consolidation,2010,Joint App,Not Verified
23,36,MORTGAGE,Source Verified,Current,car,2004,Joint App,Source Verified
25,60,MORTGAGE,Source Verified,Current,debt_consolidation,1999,Joint App,Source Verified
39,60,MORTGAGE,Not Verified,Current,credit_card,1998,Joint App,Not Verified


In [20]:
loan[loan['hardship_reason'].notnull()].iloc[:5,50:111].select_dtypes(include=['object'])

,sec_app_earliest_cr_line,hardship_flag,hardship_type,hardship_reason,hardship_status,hardship_start_date,hardship_end_date
95385,2007,Y,INTEREST ONLY-3 MONTHS DEFERRAL,UNEMPLOYMENT,ACTIVE,2019,2019
130968,2003,N,INTEREST ONLY-3 MONTHS DEFERRAL,NATURAL_DISASTER,COMPLETED,2018,2018
132095,NaN,N,INTEREST ONLY-3 MONTHS DEFERRAL,NATURAL_DISASTER,COMPLETED,2018,2018
148481,NaN,N,INTEREST ONLY-3 MONTHS DEFERRAL,NATURAL_DISASTER,COMPLETED,2018,2018
192074,NaN,N,INTEREST ONLY-3 MONTHS DEFERRAL,NATURAL_DISASTER,COMPLETED,2018,2019


In [21]:
loan[loan['hardship_loan_status'].notnull()].iloc[:5,111:].select_dtypes(include=['object'])

,payment_plan_start_date,hardship_loan_status,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date
95385,2019,Late (16-30 days),Cash,N,NaN,NaN,NaN
130968,2018,Issued,Cash,N,NaN,NaN,NaN
132095,2018,Issued,Cash,N,NaN,NaN,NaN
148481,2018,Issued,Cash,N,NaN,NaN,NaN
192074,2018,Current,Cash,N,NaN,NaN,NaN


In [22]:
loan[loan['settlement_date'].notnull()].iloc[:5,111:].select_dtypes(include=['object'])

,payment_plan_start_date,hardship_loan_status,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date
92796,NaN,NaN,Cash,Y,2019,ACTIVE,2019
116624,NaN,NaN,Cash,Y,2019,ACTIVE,2019
143914,NaN,NaN,Cash,Y,2019,ACTIVE,2019
155735,NaN,NaN,Cash,Y,2018,ACTIVE,2018
182418,NaN,NaN,Cash,Y,2019,ACTIVE,2019


[Convert](https://www.ritchieng.com/pandas-changing-datatype/) term from object to integer

In [23]:
loan.term = loan.term.astype(int)

[Convert](https://www.interviewqs.com/ddi_code_snippets/extract_month_year_pandas) columns in month_date_ls list from object to year

In [24]:
# loan[month_date_ls] = loan[month_date_ls].astype('datetime64[ns]') 

In [25]:
# loan['earliest_cr_line']

Below we check to see what the column names are for the categorical data

In [26]:
loan.select_dtypes(include=['object']).columns.tolist()

['home_ownership',
 'verification_status',
 'loan_status',
 'purpose',
 'earliest_cr_line',
 'application_type',
 'verification_status_joint',
 'sec_app_earliest_cr_line',
 'hardship_flag',
 'hardship_type',
 'hardship_reason',
 'hardship_status',
 'hardship_start_date',
 'hardship_end_date',
 'payment_plan_start_date',
 'hardship_loan_status',
 'disbursement_method',
 'debt_settlement_flag',
 'debt_settlement_flag_date',
 'settlement_status',
 'settlement_date']

Most Machine Learning Algorithms cannot work with categorical variables directly, they need to be converted to numbers.

So let's change some of the categorical data to numeric.  See [One-hot encoding](https://towardsdatascience.com/one-hot-encoding-multicollinearity-and-the-dummy-variable-trap-b5840be3c41a) or the [video](https://www.youtube.com/watch?v=PJ3E45AopM8)

We'll use scikit-learn

In [27]:
from sklearn.preprocessing import LabelEncoder 
le=LabelEncoder() 
loan.verification_status = le.fit_transform(loan.verification_status.values) 
loan.home_ownership=le.fit_transform(loan.home_ownership.values) 
loan.loan_status=le.fit_transform(loan.loan_status.values) 
loan.purpose=le.fit_transform(loan.purpose.values)

In [28]:
loan.select_dtypes(include=['object']).columns.tolist()

['earliest_cr_line',
 'application_type',
 'verification_status_joint',
 'sec_app_earliest_cr_line',
 'hardship_flag',
 'hardship_type',
 'hardship_reason',
 'hardship_status',
 'hardship_start_date',
 'hardship_end_date',
 'payment_plan_start_date',
 'hardship_loan_status',
 'disbursement_method',
 'debt_settlement_flag',
 'debt_settlement_flag_date',
 'settlement_status',
 'settlement_date']

Let us drop the remaining categorical columns

In [29]:
loan.drop(['earliest_cr_line',
 'application_type',
 'verification_status_joint',
 'sec_app_earliest_cr_line',
 'hardship_flag',
 'hardship_type',
 'hardship_reason',
 'hardship_status',
 'hardship_start_date',
 'hardship_end_date',
 'payment_plan_start_date',
 'hardship_loan_status',
 'disbursement_method',
 'debt_settlement_flag',
 'debt_settlement_flag_date',
 'settlement_status',
 'settlement_date'], axis=1, inplace=True)

In [30]:
loan[loan.isna().any(axis=1)]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,...,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
0,2500,2500,2500.0,36,13.56,84.92,5,55000.0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30000,30000,30000.0,60,18.94,777.23,1,90000.0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5000,5000,5000.0,36,17.97,180.69,1,59280.0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4000,4000,4000.0,36,18.94,146.51,1,92000.0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30000,30000,30000.0,60,16.14,731.78,1,57250.0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260663,12000,12000,12000.0,60,14.08,279.72,1,58000.0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2260664,12000,12000,12000.0,60,25.82,358.01,1,30000.0,0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2260665,10000,10000,10000.0,36,11.99,332.10,4,64000.0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2260666,12000,12000,12000.0,60,21.45,327.69,5,60000.0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can see above that we still have a lot of [NaN values](https://stackoverflow.com/questions/43424199/display-rows-with-one-or-more-nan-values-in-pandas-dataframe).  


We can use [scikit-learn](https://scikit-learn.org/stable/auto_examples/impute/plot_missing_values.html?highlight=imputing) or [pandas](https://stackoverflow.com/questions/33660836/impute-entire-dataframe-all-columns-using-scikit-learn-sklearn-without-itera) to impute [missing values](https://stackoverflow.com/questions/33113947/using-scikit-learn-sklearn-how-to-handle-missing-data-for-linear-regression).

In [31]:
from sklearn.impute import SimpleImputer
imputed_loan = loan.copy()


In [32]:
fill_NaN = SimpleImputer(missing_values=np.nan, strategy='mean')
imputed_loan = pd.DataFrame(fill_NaN.fit_transform(loan))
imputed_loan.columns = loan.columns
imputed_loan.index = loan.index

In [33]:
imputed_loan[imputed_loan.isna().any(axis=1)]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,...,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term


Looks like it worked.  To verify let's spot check

In [34]:
loan[loan['settlement_amount'].notna()].iloc[:,106:]

,settlement_amount,settlement_percentage,settlement_term
92796,5443.00,65.00,18.0
116624,10119.00,65.00,18.0
143914,23506.00,65.00,18.0
155735,3422.00,65.01,18.0
182418,13175.00,65.00,18.0
...,...,...,...
2260389,3256.00,50.01,15.0
2260402,6370.01,40.00,1.0
2260425,4954.00,50.00,16.0
2260570,2710.17,45.00,24.0


In [35]:
loan['settlement_amount'].describe()

count    33056.000000
mean      5030.606922
std       3692.027842
min         44.210000
25%       2227.000000
50%       4172.855000
75%       6870.782500
max      33601.000000
Name: settlement_amount, dtype: float64

In [36]:
imputed_loan.iloc[:5,106:]

,settlement_amount,settlement_percentage,settlement_term
0,5030.606922,47.7756,13.148596
1,5030.606922,47.7756,13.148596
2,5030.606922,47.7756,13.148596
3,5030.606922,47.7756,13.148596
4,5030.606922,47.7756,13.148596


Ok Great.  Let's now get our $X$ and $y$.  

The column named **int_rate** is the interest rate and is thus our dependent variable ($y$).  The other variables will be our independent variables ($X$).

In [37]:
imputed_loan.shape

(2260668, 109)

In [38]:
y=imputed_loan.int_rate.values

# remove y from the dataframe to get X
imputed_loan.drop(['int_rate'], axis=1, inplace=True)
imputed_loan.shape

(2260668, 108)

In [39]:
X = imputed_loan.values

In [40]:
from sklearn.model_selection import train_test_split

# the train test split
X_train, X_test, y_train, y_test = train_test_split(X,y)

Now fit a linear regression model to the trainig set and print out the results

In [41]:
from sklearn.linear_model import LinearRegression

linr=LinearRegression().fit(X_train, y_train)

print("Coefficients (theta_1..theta_n)")
print(linr.coef_)
print()
print("Y Intercept(theta0)")
print(linr.intercept_)
print()

print("R-squared for Train: %.2f" %linr.score(X_train, y_train))
print("R-squared for Test: %.2f" %linr.score(X_test, y_test))

Coefficients (theta_1..theta_n)
[-2.48241379e-04 -1.02833764e-03  8.97272620e-05  3.95871272e-01
  4.06157465e-02  9.61253402e-02 -1.64072097e-06  4.38695811e-01
  2.01917252e-01  2.18531908e-01  1.51051119e-02  1.76109668e-01
  3.81773682e-01 -5.60409826e-03 -2.17788295e-03  1.21598633e-01
  7.23400500e-02  3.00125756e-07  1.33763346e-02 -3.09220337e-02
 -6.54913775e-03  6.51941319e-03 -3.38509525e+00 -3.39573551e-05
  3.38496761e+00  3.38557279e+00  3.38067926e+00  3.38526917e+00
 -8.52829030e-04  3.40026258e-01  5.85660640e-04  2.39208643e-07
  2.75724569e-02  3.11326091e-01  1.10586938e-06  1.63373133e-06
  1.13992699e-02  4.88644164e-04  8.17360794e-02  3.93605721e-02
  5.21626858e-04 -7.96347140e-07  2.18331094e-03 -1.07853725e-01
  3.20809025e-02 -1.01138923e-05  1.25459344e-02 -5.92814788e-06
  1.15696698e-01 -2.15033800e-02  1.74390314e-03  7.79050085e-02
 -1.54602546e-05  2.05344841e-06 -1.71965850e-05  1.98400946e-01
  6.03050651e-06 -2.46237288e-03 -2.13467364e-03  1.510501

There we have it, the R-squared value on the test set is about 66%, which is not great but understandable considering 
the data must be much more sophisticated than a straight line
and we didn't do much to select our features.

Another thing we can do with this regressor is 
to normalize the data before training (value - mean /std) so all values are in the same range from 0 to 1. 
Let's try this in the next step 

In [45]:
linr=LinearRegression(normalize=True).fit(X_train, y_train)

print("Coefficients (theta_1..theta_n)")
print(linr.coef_)
print()
print("Y Intercept(theta0)")
print(linr.intercept_)
print()

print("R-squared for Train: %.2f" %linr.score(X_train, y_train))
print("R-squared for Test: %.2f" %linr.score(X_test, y_test))

Coefficients (theta_1..theta_n)
[-2.48241379e-04 -1.02833764e-03  8.97272620e-05  3.95871272e-01
  4.06157465e-02  9.61253402e-02 -1.64072097e-06  4.38695811e-01
  2.01917252e-01  2.18531908e-01  1.51051119e-02  1.76109668e-01
  3.81773682e-01 -5.60409826e-03 -2.17788295e-03  1.21598633e-01
  7.23400500e-02  3.00125756e-07  1.33763346e-02 -3.09220337e-02
 -6.54913775e-03  6.51941319e-03 -3.38509525e+00 -3.39573551e-05
  3.38496761e+00  3.38557279e+00  3.38067926e+00  3.38526917e+00
 -8.52829030e-04  3.40026258e-01  5.85660640e-04  2.39208643e-07
  2.75724569e-02  3.11326091e-01  1.10586938e-06  1.63373133e-06
  1.13992699e-02  4.88644165e-04  8.17360794e-02  3.93605721e-02
  5.21626858e-04 -7.96347141e-07  2.18331094e-03 -1.07853725e-01
  3.20809025e-02 -1.01138923e-05  1.25459344e-02 -5.92814788e-06
  1.15696698e-01 -2.15033800e-02  1.74390314e-03  7.79050085e-02
 -1.54602546e-05  2.05344841e-06 -1.71965850e-05  1.98400946e-01
  6.03050652e-06 -2.46237288e-03 -2.13467364e-03  1.510501

We could examine the date more carefully to do a better job.  Maybe we should select a different set of features or impute the data differently.  Or maybe we need another modeling approach.